<a href="https://colab.research.google.com/github/tobieabel/robokeeper/blob/master/TF2_Object_Detection_from_Checkpoint_Efficientdet_construction_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TF2 OD API - training tutorial


In this Colab notebook, we show how to train (fine-tune) a pre-trained model that is  available in [TensorFlow 2 Detection Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md). The model is trained on construction safety equipment dataset that is given as courtesy of Forsight.

We are also open-sourcing our train/eval script that enables you to simultaneously run and evaluate models without having to manually stop and reset the training.






In [ ]:
# Depending on your hardware specs install either tensorflow (CPU) or tensorflow_gpu (GPU) version

# In this notebook, we use tensorflow_gpu. Make sure that GPU hardware acceleration is enabled in your Google Colab notebook
# by checking "Runtime -> Change runtime type" and selecting "GPU accelerator"
!pip install tensorflow-gpu

# Install Trains open-source experiment management tool that will help us in experiment management.
!pip install trains

     |████████████████████████████████| 394.3MB 41kB/s 
     |████████████████████████████████| 860kB 4.1MB/s 
     |████████████████████████████████| 92kB 12.7MB/s 


In [ ]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist.
# We clone from a forked repo because the official tf repo doesn't support per object
# statistics in eval, and it doesn't support simultaneous train and eval. 
# Take a look into this issue on TF OD API git for additional details:
# https://github.com/tensorflow/models/issues/4778#issuecomment-430262110

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/qraleq/models

# By default the cloned repository is located in /content/models/
# The TF OD API part of the repo is located in /content/models/research/object_detection/
# Usually, all the scripts are ran from /content/models/research/ as the base directory
# In order to explore the scripts in the OD API, you can use the Files navigation bar
# on the left side of the Google Colab notebook.
# Double-clicking a file will open a scratch editor on the right side of the Google Colab notebook.

%cd /content/models/research

# COCO is a large image dataset designed for object detection, segmentation, person keypoints detection, stuff segmentation, and caption generation. 
# TF 2 OD API uses PythonAPI pycocotools for performing evaluation of the object detection models.
# By default, pycocotools do not support per object statistic in eval, so we replace cocoeval.py script with a custom one that does support it.
!cp ./object_detection/metrics/cocoeval.py /usr/local/lib/python3.6/dist-packages/pycocotools/

Cloning into 'models'...
remote: Enumerating objects: 1968, done.
remote: Counting objects: 100% (1968/1968), done.
remote: Compressing objects: 100% (1686/1686), done.
remote: Total 1968 (delta 471), reused 879 (delta 262), pack-reused 0
Receiving objects: 100% (1968/1968), 51.42 MiB | 16.74 MiB/s, done.
Resolving deltas: 100% (471/471), done.
/content/models/research


In [ ]:
# Compile protos.
!protoc object_detection/protos/*.proto --python_out=.

# Export OD API to Python path and install dependencies
os.environ['PYTHONPATH'] += ":/content/models/research:/content/models/research/slim:/content/models/research/slim/nets:/content/models/research/object_detection"
!pip install tf_slim tf-models-official

     |████████████████████████████████| 358kB 5.7MB/s 
     |████████████████████████████████| 1.1MB 32.2MB/s 
     |████████████████████████████████| 37.6MB 84kB/s 
     |████████████████████████████████| 174kB 60.5MB/s 
     |████████████████████████████████| 1.2MB 58.3MB/s 
     |████████████████████████████████| 645kB 55.7MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
     |████████████████████████████████| 102kB 13.5MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp36-none-any.whl size=16171 sha256=159ac1df98247032b8e6482b7b1eedaaabdbf472951eafa43500fc5a29337069
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
  Created wheel for py-cpuinfo: filename=py_cpuinfo-7.0.0-cp36-none-any.whl size=20072 sha256=0957180794eb83731aa8f62931b829d4d4018a034891e87e870ce356276fac41
  Stored in directory: /root/.cache/pip/wheels/f1/93/7b/127daf0c3a5a49feb2fecd468d508067c733fba5192f726ad1
Successfully built seqeval py-

In [ ]:
# Download EfficientDet D0 checkpoint and uncompress it
%cd ./object_detection/models/checkpoints/detection/

# We use a simple bash script that downloads and uncompresses the EfficientDet
# checkpoint for us
!bash download_d0_efficientdet_checkpoints.sh

/content/models/research/object_detection/models/checkpoints/detection
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 29.3M  100 29.3M    0     0  13.8M      0  0:00:02  0:00:02 --:--:-- 13.8M


In [ ]:
# Download and prepare the dataset and label map
%cd /content
!gdown --id 1GPU-RK-1GuCtbQzV6YQDRRX039HiQiBC
!unzip tf_records.zip
!ls tf_records

/content
Downloading...
From: https://drive.google.com/uc?id=1GPU-RK-1GuCtbQzV6YQDRRX039HiQiBC
To: /content/tf_records.zip
265MB [00:04, 54.2MB/s]
Archive:  tf_records.zip
   creating: tf_records/
  inflating: tf_records/construction_label_map.pbtxt  
  inflating: tf_records/test.rcd-00000-of-00010  
  inflating: tf_records/test.rcd-00001-of-00010  
  inflating: tf_records/test.rcd-00002-of-00010  
  inflating: tf_records/test.rcd-00003-of-00010  
  inflating: tf_records/test.rcd-00004-of-00010  
  inflating: tf_records/test.rcd-00005-of-00010  
  inflating: tf_records/test.rcd-00006-of-00010  
  inflating: tf_records/test.rcd-00007-of-00010  
  inflating: tf_records/test.rcd-00008-of-00010  
  inflating: tf_records/test.rcd-00009-of-00010  
  inflating: tf_records/train.rcd-00000-of-00010  
  inflating: tf_records/train.rcd-00001-of-00010  
  inflating: tf_records/train.rcd-00002-of-00010  
  inflating: tf_records/train.rcd-00003-of-00010  
  inflating: tf_records/train.rcd-00004-of-0

In [ ]:
# Update the pipeline.config file for the EfficientDet checkpoint.
%cd /content/models/research

from object_detection.utils import config_util

configs = config_util.get_configs_from_pipeline_file(f'./object_detection/models/checkpoints/detection/efficientdet_d0_coco17_tpu-32/pipeline.config')

# Modify the model config
model_config = configs['model'].ssd
model_config.num_classes = 4

train_config = configs['train_config']
train_config.fine_tune_checkpoint = f'./object_detection/models/checkpoints/detection/efficientdet_d0_coco17_tpu-32/checkpoint/ckpt-0'
train_config.fine_tune_checkpoint_type = 'detection'
train_config.batch_size = 16

train_input_reader_config = configs['train_input_config']
train_input_reader_config.label_map_path = '/content/tf_records/construction_label_map.pbtxt'
train_input_reader_config.tf_record_input_reader.input_path[:] = ['/content/tf_records/train.rcd-?????-of-?????']

# Modify the eval_configs
eval_config = configs['eval_config']
eval_config.num_visualizations = 20 # total number of images for visualization
eval_config.max_evals = 1 # total number of evals
eval_config.include_metrics_per_category = True # include per category metrics
eval_config.all_metrics_per_category = True # include detailed per category metrics

# Modify the eval_input_configs
eval_input_reader_config = configs['eval_input_config']
eval_input_reader_config.label_map_path = '/content/tf_records/construction_label_map.pbtxt' # path to the label map
eval_input_reader_config.tf_record_input_reader.input_path[:] = ['/content/tf_records/test.rcd-?????-of-?????']
eval_input_reader_config.num_readers = 8

# Convert config to pipeline proto and save to file
pipeline_proto = config_util.create_pipeline_proto_from_configs(configs)
config_util.save_pipeline_config(pipeline_config=pipeline_proto, directory=f'./object_detection/models/checkpoints/detection/efficientdet_d0_coco17_tpu-32/')
!cat './object_detection/models/checkpoints/detection/efficientdet_d0_coco17_tpu-32/pipeline.config'

/content/models/research
INFO:tensorflow:Writing pipeline config file to ./object_detection/models/checkpoints/detection/efficientdet_d0_coco17_tpu-32/pipeline.config
model {
  ssd {
    num_classes: 4
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 512
        max_dimension: 512
        pad_to_max_dimension: true
      }
    }
    feature_extractor {
      type: "ssd_efficientnet-b0_bifpn_keras"
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 4e-05
          }
        }
        initializer {
          truncated_normal_initializer {
            mean: 0.0
            stddev: 0.03
          }
        }
        activation: SWISH
        batch_norm {
          decay: 0.99
          scale: true
          epsilon: 0.001
        }
        force_use_bias: true
      }
      bifpn {
        min_level: 3
        max_level: 7
        num_iterations: 3
        num_filters: 64
      }
    }
    box_coder {
      faster_rcnn_

In [ ]:
!python3 model_main_tf2.py \
--model_dir=trained_model \
--pipeline_config_path=efficientdet_d0_coco17_tpu-32/pipeline.config \
--alsologtostderr \

In [ ]:
cd ../

/


## Adding Allegro Trains to TF2 OD API

After we've installed [Allegro Trains](https://allegro.ai/trains-open-source/?utm_source=i_blog&utm_medium=referral&utm_campaign=trains_c)  using `pip install trains`, it's pretty straightforward to start using it within the TF2 OD API. For detailed documentation on Allegro Trains and all its functionalities take a look [here](https://allegro.ai/docs/). 


In the `model_main_tf2.py` script we've added these lines:

1. `from trains import Task` - import Task class from trains



> The Task class is the Trains Python Client package multipurpose class which supports experimentation and various workflows. In experimentation, a Task object connects your experiment code to Trains Server, where Trains stores it. All the parts of an experiment connect to a Task. This includes models, hyperparameters, and logging. A Task is, effectively, an experiment in Trains. Once it is stored in Trains Server, you can rerun the Task (experiment), reproduce it, and tune it.


2. `task = Task.init(project_name="TF2 OD API - Evaluation of EfficientDet", task_name=PurePath(FLAGS.pipeline_config_path).parent.name)` - define the project name and task name that will be shown in the Trains dashboard



> Trains Automatically Logs Everything - with only two lines of code, you're getting:
* Git repository, branch, commit id, entry point and local git diff
* Python environment (including specific packages & versions)
* stdout and stderr
* Resource Monitoring (CPU/GPU utilization, temperature, IO, network, etc.)
* Hyper-parameters
    * ArgParser for command line parameters with currently used values
    * Explicit parameters dictionary
    * Tensorflow Defines (absl-py)
* Initial model weights file
* Model snapshots (With optional automatic upload to central storage: Shared folder, S3, GS, Azure, Http)
* Artifacts log & store (Shared folder, S3, GS, Azure, Http)
* Tensorboard/TensorboardX scalars, metrics, histograms, **images, audio and video**
* [Matplotlib & Seaborn](https://github.com/allegroai/trains/tree/master/examples/frameworks/matplotlib)
* Supported frameworks: [PyTorch](https://github.com/allegroai/trains/tree/master/examples/frameworks/pytorch), [Tensorflow](https://github.com/allegroai/trains/tree/master/examples/frameworks/tensorflow), [Keras](https://github.com/allegroai/trains/tree/master/examples/frameworks/keras), [AutoKeras](https://github.com/allegroai/trains/tree/master/examples/frameworks/autokeras), [XGBoost](https://github.com/allegroai/trains/tree/master/examples/frameworks/xgboost) and [Scikit-Learn](https://github.com/allegroai/trains/tree/master/examples/frameworks/scikit-learn) (MxNet is coming soon)
* Seamless integration (including version control) with **Jupyter Notebook**
    and [*PyCharm* remote debugging](https://github.com/allegroai/trains-pycharm-plugin)    



3. `task.connect_configuration(FLAGS.pipeline_config_path)` - OPTIONAL: connect `pipeline.config` file so it's logged and shown in the Trains dashboard

and after running the training script, we got all the eval experiments logged in the Trains dashboard at https://demoapp.trains.allegro.ai under TF2 OD API - Training Tutorial.


In [ ]:
!pwd
!ls /content/
%cd /content/models/research

import random
import string

PIPELINE_CONFIG_PATH='./object_detection/models/checkpoints/detection/efficientdet_d0_coco17_tpu-32/pipeline.config'
NUM_TRAIN_STEPS=20000
CHECKPOINT_EVERY_N=250
EVAL_EVERY_N_SECONDS=2000
SAMPLE_1_OF_N_EVAL_EXAMPLES=1
PROJECT_NAME='\TF2\ OD\ API\ -\ Training\ Tutorial'
TASK_NAME=''.join(random.choices(string.ascii_lowercase + string.digits, k=8))
MODEL_DIR='/tmp/model_outputs_'+TASK_NAME


print("Please find your results at https://demoapp.trains.allegro.ai with \nPROJECT_NAME: " + PROJECT_NAME.replace("\\", "") + "\nTASK_NAME: " + TASK_NAME)

/content
sample_data
[Errno 2] No such file or directory: '/content/models/research'
/content
Please find your results at https://demoapp.trains.allegro.ai with 
PROJECT_NAME: TF2 OD API - Training Tutorial
TASK_NAME: ovyr0xl3


In [ ]:
!python ./object_detection/model_main_tf2_train_eval.py --model_dir=$MODEL_DIR --num_train_steps=$NUM_TRAIN_STEPS --sample_1_of_n_eval_examples=$SAMPLE_1_OF_N_EVAL_EXAMPLES --pipeline_config_path=$PIPELINE_CONFIG_PATH --project_name=$PROJECT_NAME --task_name=$TASK_NAME --eval_time=$EVAL_EVERY_N_SECONDS --checkpoint_every_n=$CHECKPOINT_EVERY_N

python3: can't open file './object_detection/model_main_tf2_train_eval.py': [Errno 2] No such file or directory


# Testing the trained model on sample images

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

%matplotlib inline

def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: the file path to the image

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
pipeline_config = PIPELINE_CONFIG_PATH
model_dir = MODEL_DIR

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
detection_model = model_builder.build(
      model_config=model_config, is_training=False)

for latest_checkpoint in tf.train.checkpoints_iterator(model_dir, timeout=1):
  ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)

ckpt.restore(latest_checkpoint).expect_partial()

def get_model_detection_function(model):
  """Get a tf.function for detection."""

  @tf.function
  def detect_fn(image):
    """Detect objects in image."""

    image, shapes = model.preprocess(image)
    prediction_dict = model.predict(image, shapes)
    detections = model.postprocess(prediction_dict, shapes)

    return detections, prediction_dict, tf.reshape(shapes, [-1])

  return detect_fn

detect_fn = get_model_detection_function(detection_model)

In [ ]:
label_map_path = configs['eval_input_config'].label_map_path
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(
    label_map,
    max_num_classes=label_map_util.get_max_label_map_index(label_map),
    use_display_name=True)
category_index = label_map_util.create_category_index(categories)
label_map_dict = label_map_util.get_label_map_dict(label_map, use_display_name=True)

In [ ]:
!curl "https://images.pexels.com/photos/3680959/pexels-photo-3680959.jpeg?crop=entropy&cs=srgb&dl=pexels-aleksey-3680959.jpg&fit=crop&fm=jpg&h=426&w=640" --output "/content/models/research/object_detection/test_images/construction2.jpg"
image_dir = '/content/models/research/object_detection/test_images/'
image_path = os.path.join(image_dir, 'construction2.jpg')
image_np = load_image_into_numpy_array(image_path)


input_tensor = tf.convert_to_tensor(
    np.expand_dims(image_np, 0), dtype=tf.float32)
detections, predictions_dict, shapes = detect_fn(input_tensor)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections,
      detections['detection_boxes'][0].numpy(),
      (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
      detections['detection_scores'][0].numpy(),
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=20,
      min_score_thresh=.80)

plt.figure(figsize=(12,16))
plt.imshow(image_np_with_detections)
plt.show()